<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-08 09:31:26
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.34 C
-------------------
Today PnL: 9.88 K (0.07%)
Current PnL: -21.30 L (-14.49%)
CY Booked + Current PnL: -13.03 L (-8.86%)
-------------------
Total profit:  2.18 L
Total loss:  -23.48 L
-------------------
Total Booked + Current PnL: 13.95 L (11.62%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.19%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 79.56 L (59.56%)
Deployed:  1.20 C
Current:  1.34 C
CAGR/XIRR %: 6.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-20.73,H-LC,5.71,109232.0,4920.0,11732.0,0.13,4.72,10.74,15.96,10.0,0.42,0.84,22.41,X5K,ATH,METALS
71,SIS,477.00,477.00,2384.46,H-SC,7.38,66844.0,-14246.0,14244.0,5.84,-17.57,21.31,0.00,157.0,-1.00,0.51,35.47,OX40N,NTT,MISC
67,SBIN,760.30,863.00,-11.51,M-LC,3.93,213510.0,12030.0,15181.0,0.07,5.97,7.11,13.51,62.0,0.79,1.64,19.32,XY25,NTT,BANKS
82,TTKPRESTIG,769.29,770.00,-8.69,H-SC,11.14,83558.0,-17219.0,17313.0,0.60,-17.09,20.72,0.09,145.0,-0.99,0.64,9.78,OX40N,NTT,DURABLES
27,DIXON,14367.00,18940.15,-27.61,H-MC,6.63,133304.0,18368.0,18223.0,0.91,15.98,13.67,31.83,85.0,1.01,1.03,45.53,X40N,ATH,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,NATIONALUM,189.63,247.44,-50.15,H-MC,2.81,100573.0,69.0,30574.0,0.60,0.07,30.40,30.49,81.0,0.00,0.77,32.50,MH,ATH,METALS
91,WIPRO,243.46,420.00,-12.39,M-LC,6.47,149953.0,-992.0,110440.0,-0.19,-0.66,73.65,72.51,55.0,-0.01,1.15,5.33,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.84,H-LC,1.92,203157.0,-1343.0,42765.0,-0.54,-0.66,21.05,20.25,9.0,-0.03,1.56,8.02,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-14.41,H-LC,2.47,170095.0,-31291.0,118896.0,-0.20,-15.54,69.90,43.51,19.0,-0.26,1.31,18.40,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-6.24,H-LC,3.53,180666.0,-19644.0,79620.0,-0.38,-9.81,44.07,29.94,24.0,-0.25,1.39,24.81,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-10.62,H-LC,3.63,185550.0,-14956.0,63736.0,0.10,-7.46,34.35,24.33,30.0,-0.23,1.43,7.00,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-59.03,H-LC,4.14,129847.0,-135579.0,192485.0,-0.67,-51.08,148.24,21.44,26.0,-0.70,1.00,5.01,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,477.00,477.0,2384.46,H-SC,7.38,66844.0,-14246.0,14244.0,5.84,-17.57,21.31,0.00,157.0,-1.00,0.51,35.47,OX40N,NTT,MISC
23,COFFEEDAY,59.14,80.0,-51.71,L-SC,22.03,70541.0,-43008.0,83062.0,3.67,-37.88,117.75,35.27,267.0,-0.52,0.54,72.65,XR,NTT,HOTELS
83,UJJIVANSFB,52.77,53.0,38.86,H-SC,20.70,120231.0,-22248.0,22868.0,2.98,-15.61,19.02,0.44,163.0,-0.97,0.92,41.91,OX40N,NTT,BANKS
62,REPCOHOME,515.07,880.0,-57.53,H-SC,3.89,157727.0,-47271.0,192506.0,2.23,-23.06,122.05,70.85,146.0,-0.25,1.21,27.33,XY24,NTT,FINANCE
8,AWL,325.39,485.0,-69.74,H-MC,5.24,210292.0,-58155.0,189831.0,1.68,-21.66,90.27,49.05,82.0,-0.31,1.62,8.12,XY24,NTT,FMCG


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,BSOFT,628.85,836.99,-22.90,H-SC,13.75,64872.0,-45177.0,81602.0,-2.95,-41.05,125.79,33.10,129.0,-0.55,0.50,7.80,XR,ATH,IT
15,BLUESTARCO,1646.70,2326.38,-4.81,H-SC,7.97,178500.0,13830.0,54139.0,-2.33,8.40,30.33,41.28,119.0,0.26,1.37,17.05,X40N,ATH,AC
52,LAOPALA,369.40,464.00,121.82,H-SC,3.43,70461.0,-30385.0,56214.0,-2.33,-30.13,79.78,25.61,141.0,-0.54,0.54,30.64,AR,NTT,CERAMICS
42,INDIAMART,2327.09,4850.92,-48.45,H-SC,13.28,134228.0,10892.0,122872.0,-1.08,8.83,91.54,108.45,118.0,0.09,1.03,33.16,AR,ATH,MISC
80,TITAGARH,1097.23,1548.00,-9.77,H-SC,5.99,147875.0,-44140.0,123032.0,-1.05,-22.99,83.20,41.08,158.0,-0.36,1.14,25.16,XY24,NTT,ENGINEERING


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,3.89,148800.0,-25759.0,25787.0,-0.83,-14.76,17.33,0.02,135.0,-1.00,1.14,29.03,OX40N,NTT,PAINTS
83,UJJIVANSFB,52.77,53.0,38.86,H-SC,20.70,120231.0,-22248.0,22868.0,2.98,-15.61,19.02,0.44,163.0,-0.97,0.92,41.91,OX40N,NTT,BANKS
82,TTKPRESTIG,769.29,770.0,-8.69,H-SC,11.14,83558.0,-17219.0,17313.0,0.60,-17.09,20.72,0.09,145.0,-0.99,0.64,9.78,OX40N,NTT,DURABLES
71,SIS,477.00,477.0,2384.46,H-SC,7.38,66844.0,-14246.0,14244.0,5.84,-17.57,21.31,0.00,157.0,-1.00,0.51,35.47,OX40N,NTT,MISC
77,TATAELXSI,7332.28,7332.0,-22.02,H-MC,2.12,81900.0,-20752.0,20745.0,-0.26,-20.22,25.33,-0.00,101.0,-1.00,0.63,24.50,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.06,H-LC,15.37,237175.0,-58759.0,152670.0,-0.21,-19.86,64.37,31.73,1.0,-0.38,1.82,1.26,X40,BTT,IT
88,VBL,492.64,671.64,-10.92,H-LC,11.27,270453.0,5413.0,90899.0,0.70,2.04,33.61,36.33,3.0,0.06,2.08,15.61,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-20.32,H-LC,9.50,255182.0,-6419.0,152037.0,-0.79,-2.45,59.58,55.67,4.0,-0.04,1.96,3.50,X40,BTT,IT
1,ABB,5551.76,7934.00,-44.15,H-LC,9.40,199232.0,-17287.0,110195.0,-0.00,-7.98,55.31,42.91,5.0,-0.16,1.53,4.34,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,0.97,H-LC,9.91,179352.0,-11294.0,57805.0,-0.37,-5.92,32.23,24.40,8.0,-0.20,1.38,9.07,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,20.18,H-SC,13.41,153932.0,27732.0,18441.0,-0.86,21.97,11.98,36.59,156.0,1.5,1.18,37.24,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,25.90,H-SC,13.23,128797.0,3500.0,55576.0,-0.32,2.79,43.15,47.15,144.0,0.06,0.99,61.13,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.41,153932.0,27732.0,18441.0,-0.86,21.97,11.98,36.59,156.0,1.50,1.18,37.24,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,1.90,H-MC,6.11,163821.0,12908.0,44395.0,0.43,8.55,27.10,37.97,90.0,0.29,1.26,36.02,AR,ATH,AUTO
15,BLUESTARCO,1646.70,2326.38,-4.81,H-SC,7.97,178500.0,13830.0,54139.0,-2.33,8.40,30.33,41.28,119.0,0.26,1.37,17.05,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-5.03,H-SC,2.36,106719.0,4715.0,39209.0,0.09,4.62,36.74,43.06,120.0,0.12,0.82,22.53,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,1.90,H-MC,6.11,163821.0,12908.0,44395.0,0.43,8.55,27.10,37.97,90.0,0.29,1.26,36.02,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-48.45,H-SC,13.28,134228.0,10892.0,122872.0,-1.08,8.83,91.54,108.45,118.0,0.09,1.03,33.16,AR,ATH,MISC
90,WHIRLPOOL,1219.98,2270.00,-42.72,M-SC,4.87,96390.0,4892.0,73864.0,-0.73,5.35,76.63,86.07,211.0,0.07,0.74,39.44,XR,NTT,DURABLES
50,KPIGREEN,497.21,731.64,25.90,H-SC,13.23,128797.0,3500.0,55576.0,-0.32,2.79,43.15,47.15,144.0,0.06,0.99,61.13,MH,ATH,POWER
56,NATIONALUM,189.63,247.44,-50.15,H-MC,2.81,100573.0,69.0,30574.0,0.60,0.07,30.40,30.49,81.0,0.00,0.77,32.50,MH,ATH,METALS


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,477.00,477.00,2384.46,H-SC,7.38,66844.0,-14246.0,14244.0,5.84,-17.57,21.31,0.00,157.0,-1.00,0.51,35.47,OX40N,NTT,MISC
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,22.03,70541.0,-43008.0,83062.0,3.67,-37.88,117.75,35.27,267.0,-0.52,0.54,72.65,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-2.49,H-SC,17.99,90713.0,-10250.0,104783.0,0.06,-10.15,115.51,93.63,149.0,-0.10,0.70,35.11,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-19.15,H-SC,16.07,92085.0,-5895.0,27764.0,1.32,-6.02,30.15,22.32,151.0,-0.21,0.71,33.56,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-20.73,H-LC,5.71,109232.0,4920.0,11732.0,0.13,4.72,10.74,15.96,10.0,0.42,0.84,22.41,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.41,153932.0,27732.0,18441.0,-0.86,21.97,11.98,36.59,156.0,1.50,1.18,37.24,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-23.40,H-SC,8.06,217824.0,24804.0,23242.0,-0.31,12.85,10.67,24.89,128.0,1.07,1.68,39.20,X40N,NTT,HEALTHCARE
69,SHALBY,261.39,327.00,946.25,M-SC,8.15,142335.0,-38808.0,84277.0,-0.16,-21.42,59.21,25.10,233.0,-0.46,1.09,14.07,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,1.90,H-MC,6.11,163821.0,12908.0,44395.0,0.43,8.55,27.10,37.97,90.0,0.29,1.26,36.02,AR,ATH,AUTO
88,VBL,492.64,671.64,-10.92,H-LC,11.27,270453.0,5413.0,90899.0,0.70,2.04,33.61,36.33,3.0,0.06,2.08,15.61,X40N,ATH,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.40
1,25,41.26
2,50,71.21


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.97
LC    36.50
MC    22.46
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.20
X40N     15.40
X40      13.42
XY25     10.80
XR        9.05
AR        7.77
OX40N     7.74
X5K       2.26
MH        1.76
X200      1.39
SR        1.14
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.24
H-SC    27.59
H-MC    19.46
M-SC    11.92
M-LC     7.18
M-MC     2.65
L-SC     1.46
L-LC     1.08
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.39,-10.24,45.68
IT,12.06,-18.91,76.67
FINANCE,8.39,-19.38,62.69
BANKS,7.66,-14.29,60.92
PAINTS,5.79,-16.07,38.94
MISC,5.18,-20.97,91.68
HEALTHCARE,5.16,2.20,22.50
ELECTRICAL,4.77,-13.35,62.44
AUTO,4.47,-16.10,66.62


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2837642.0,23
XR,1114223.0,13
AR,1053963.0,9
X40,825859.0,10
XY25,644107.0,8
X40N,606841.0,12
OX40N,367597.0,10
SR,195209.0,2
X5K,144466.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2731101.0,30
H-LC,1480067.0,20
M-SC,1415534.0,15
H-MC,1177270.0,15
M-LC,444218.0,5
M-MC,333406.0,2
L-SC,271180.0,3
L-MC,56660.0,1
L-LC,46241.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      945560.0      7
M-SC       XY24      861504.0      7
H-SC       AR        618556.0      4
H-LC       X40       549193.0      6
H-SC       XR        522189.0      6
H-MC       XY24      466988.0      4
M-MC       XY24      333406.0      2
H-LC       X40N      313783.0      6
           XY25      249980.0      3
           AR        231438.0      2
H-MC       XY25      210326.0      2
H-SC       X40N      209801.0      4
H-MC       X40       200731.0      3
H-SC       SR        195209.0      2
M-LC       XY24      185863.0      2
H-SC       OX40N     184210.0      6
L-SC       XR        181662.0      2
M-SC       AR        159574.0      2
M-LC       X5K       132734.0      1
M-SC       XR        123018.0      2
           XY25      122379.0      1
H-MC       XR        120254.0      1
M-LC       XR        110440.0      1
L-SC       OX40N      89518.0      1
H-MC       X40N       83257.0      2
H-LC       X200       79620.0      1
M-SC       X40        75935.0      1
           OX40N      73124.0      2
L-MC       XR         56660.0      1
H-SC       MH         55576.0      1
L-LC       XY25       46241.0      1
H-MC       AR         44395.0      1
H-LC       XY24       44321.0      1
H-MC       MH         30574.0      1
           OX40N      20745.0      1
M-LC       XY25       15181.0      1
H-LC       X5K        11732.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 57.0 seconds
